# 1 CNN

#1 CNN (MNIST)

https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/images/cnn.ipynb?hl=ko#scrollTo=jKgyC5K_4O0d

### 1) MNIST 데이터셋 다운로드하고 준비하기

In [ ]:
#!pip install tensorflow-gpu==2.0.0-rc1
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1)) #데이터 건수, 이미지 높이, 이미지 너비, 컬러 채널
test_images = test_images.reshape((10000, 28, 28, 1)) #데이터 건수, 이미지 높이, 이미지 너비, 컬러 채널

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

11493376/11490434 [==============================] - 1s 0us/step


### 2) CNN 만들기

In [ ]:
model = models.Sequential()
# 특징 추출 (Feature Extraction)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) # 32 * 3 * 3 + 32 = 320
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 32 * 64 * 3 * 3 + 64 = 18496
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) # 64 * 64 * 3 * 3 + 64 = 36928

# 분류 (Classification)
model.add(layers.Flatten()) # 576개 벡터로 Flatten
model.add(layers.Dense(64, activation='relu')) # 576 * 64 + 64 = 36928
model.add(layers.Dense(10, activation='softmax')) # 64 * 10 + 10 = 650

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

### 3) 모델 컴파일과 훈련하기

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1388 - accuracy: 0.9582
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0445 - accuracy: 0.9860
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0323 - accuracy: 0.9895
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0250 - accuracy: 0.9921
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0192 - accuracy: 0.9941


### 4) 모델 평가

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

313/313 - 1s - loss: 0.0292 - accuracy: 0.9906
0.9905999898910522


---

# 2 CNN for Sentence Classification

https://www.aclweb.org/anthology/D14-1181/

<img src="http://www.wildml.com/wp-content/uploads/2015/11/Screen-Shot-2015-11-06-at-8.03.47-AM-1024x413.png" />

## 1) 네이버 영화 리뷰 다운로드

In [ ]:
!pip install tensorflow-gpu 

In [ ]:
# 네이버 영화 리뷰 다운로드
!wget https://github.com/e9t/nsmc/raw/master/ratings.txt

import pandas as pd
import numpy as np
df = pd.read_csv("./ratings.txt",sep='\t').dropna()
df.head(5)

--2020-08-24 01:41:27--  https://github.com/e9t/nsmc/raw/master/ratings.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt [following]
--2020-08-24 01:41:27--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt.1’

ratings.txt.1       100%[===================>]  18.61M  36.2MB/s    in 0.5s    

2020-08-24 01:41:28 (36.2 MB/s) - ‘ratings.txt.1’ saved [19515078/19515078]



,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


## 2) 전처리

In [ ]:
import json
from tensorflow.keras import preprocessing

def preprocess(x, y, padding_size = 128, oov_token="<UNK>", vocab_file = "vocab.json", train_ratio=0.7) :  
  preprocessor = preprocessing.text.Tokenizer(oov_token=oov_token) #토큰화
  preprocessor.fit_on_texts(x)
  x = preprocessor.texts_to_sequences(x) #시퀀스로 변환
  vocab = preprocessor.word_index #단어:인덱스 dictionary
  json.dump(vocab, open(vocab_file, 'w'), ensure_ascii=False)
  x = preprocessing.sequence.pad_sequences(x, maxlen=padding_size, padding='post', truncating='post')

  index = int(len(x)*train_ratio)

  return x[:index], y[:index], x[index:], y[index:], vocab

x_train, y_train, x_test, y_test, vocab = preprocess(df["document"].tolist(), df["label"].tolist())

## 3) 모델 정의

<img src="https://miro.medium.com/max/770/0*wigQtmJiv0bddwPI." width="500" />

In [ ]:
from tensorflow import keras

def CNNforText( num_classes,  #클래스 갯수
          vocab_size,
          embed_size = 512, #단어 임베딩 사이즈                 
          filter_sizes = [3,4,5],
          regularizers_lambda = 0.01, #L2 regulation parameter
          dropout =  0.5,
          feature_size = 128, #문장 시퀀스 길이
          num_filters = 128 #필터 개수 (필터사이즈와 같음). mhlee 하나로 통일하자
) :
          

  inputs = keras.Input(shape=(feature_size,), name='input_data')
  embed_initer = keras.initializers.RandomUniform(minval=-1, maxval=1)
  #sequence 임베딩
  embed = keras.layers.Embedding(vocab_size, embed_size,
                                  embeddings_initializer=embed_initer,
                                  input_length=feature_size,
                                  name='embedding')(inputs)
                                  
  embed = keras.layers.Reshape((feature_size, embed_size, 1), name='add_channel')(embed)

  pool_outputs = []

  #filter 별로 모델 구성
  for filter_size in filter_sizes :
    #convolution
    filter_shape = (filter_size, embed_size)
    conv = keras.layers.Conv2D(num_filters, filter_shape, strides=(1, 1), padding='valid',
                                data_format='channels_last', activation='relu',
                                kernel_initializer='glorot_normal',
                                bias_initializer=keras.initializers.constant(0.1),
                                name='convolution_{:d}'.format(filter_size))(embed)
    #max pooling
    max_pool_shape = (feature_size - filter_size + 1, 1)
    pool = keras.layers.MaxPool2D(pool_size=max_pool_shape,
                                  strides=(1, 1), padding='valid',
                                  data_format='channels_last',
                                  name='max_pooling_{:d}'.format(filter_size))(conv)
    pool_outputs.append(pool)

  pool_outputs = keras.layers.concatenate(pool_outputs, axis=-1, name='concatenate')
  pool_outputs = keras.layers.Flatten(data_format='channels_last', name='flatten')(pool_outputs)
  pool_outputs = keras.layers.Dropout(dropout, name='dropout')(pool_outputs)

  outputs = keras.layers.Dense(num_classes, activation='softmax',
                                kernel_initializer='glorot_normal',
                                bias_initializer=keras.initializers.constant(0.1),
                                kernel_regularizer=keras.regularizers.l2(regularizers_lambda),
                                bias_regularizer=keras.regularizers.l2(regularizers_lambda),
                                name='dense')(pool_outputs)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.summary()
  return model, num_classes

model, num_classes = CNNforText(len(np.unique(y_train)), len(vocab))

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 128, 512)     188943872   input_data[0][0]                 
__________________________________________________________________________________________________
add_channel (Reshape)           (None, 128, 512, 1)  0           embedding[0][0]                  
__________________________________________________________________________________________________
convolution_3 (Conv2D)          (None, 126, 1, 128)  196736      add_channel[0][0]                
_______________________________________________________________________________________

## 4) 학습

- 상당한 RAM을 소모하므로, size를 전체적으로 줄여서 학습시킬 것

In [ ]:
import os
import time
import tensorflow as tf

def train(model, x_train, y_train, num_classes
          , batch_size = 64, epochs = 1, fraction_validation = 0.05, results_dir = "./result/", save_path = "model") :
  timestamp = time.strftime("%Y-%m-%d-%H-%M", time.localtime(time.time()))
  path = os.path.join(results_dir, timestamp)
  if not os.path.exists(path) :    
    path_log = os.path.join(path, 'log/')
    os.makedirs(path_log)

  model.compile(tf.optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
  #모델 구조 이미지 파일로 저장
  keras.utils.plot_model(model, show_shapes=True, to_file=os.path.join(path, "model.jpg"))
  y_train = tf.one_hot(y_train, num_classes)
  tb_callback = keras.callbacks.TensorBoard(path_log,
                                            histogram_freq=0.1, write_graph=True,
                                            write_images=True,
                                            embeddings_freq=0.5, update_freq='batch')

  history = model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs,
                                callbacks=[tb_callback], validation_split=fraction_validation, shuffle=True)
  
  #모델 저장
  keras.models.save_model(model, save_path)
  print(history.history)

  return model, path_log

model, path_log = train(model, x_train[:1000], y_train[:1000], num_classes, epochs=10)

Epoch 1/10
15/15 [==============================] - 70s 5s/step - loss: 0.0934 - accuracy: 0.9558 - val_loss: 0.0412 - val_accuracy: 1.0000
Epoch 2/10
15/15 [==============================] - 61s 4s/step - loss: 0.0407 - accuracy: 1.0000 - val_loss: 0.0400 - val_accuracy: 1.0000
Epoch 3/10
15/15 [==============================] - 52s 3s/step - loss: 0.0393 - accuracy: 1.0000 - val_loss: 0.0385 - val_accuracy: 1.0000
Epoch 4/10
15/15 [==============================] - 56s 4s/step - loss: 0.0377 - accuracy: 1.0000 - val_loss: 0.0367 - val_accuracy: 1.0000
Epoch 5/10
15/15 [==============================] - 59s 4s/step - loss: 0.0358 - accuracy: 1.0000 - val_loss: 0.0348 - val_accuracy: 1.0000
Epoch 6/10
15/15 [==============================] - 55s 4s/step - loss: 0.0339 - accuracy: 1.0000 - val_loss: 0.0328 - val_accuracy: 1.0000
Epoch 7/10
15/15 [==============================] - 53s 4s/step - loss: 0.0319 - accuracy: 1.0000 - val_loss: 0.0308 - val_accuracy: 1.0000
Epoch 8/10
15/15 [==

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {path_log}

## 5) 평가

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

def test(model, x_test, y_test, num_classes):
  y_pred_one_hot = model.predict(x=x_test, batch_size=1, verbose=1)
  y_pred = tf.math.argmax(y_pred_one_hot, axis=1)

  print('\nTest accuracy: {}\n'.format(accuracy_score(y_test, y_pred)))
  print('Classification report:')
  target_names = ['class {:d}'.format(i) for i in np.arange(num_classes)]
  print(classification_report(y_test, y_pred, target_names=target_names, digits=4))

test(model, x_test[:50000], y_test[:50000], num_classes)

# 3 CNN for Sentence Classification (FastText)

https://www.aclweb.org/anthology/D14-1181/

<img src="http://www.wildml.com/wp-content/uploads/2015/11/Screen-Shot-2015-11-06-at-8.03.47-AM-1024x413.png" />

## 1) 네이버 영화 리뷰 다운로드

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 3.8MB 63.5MB/s 
     |████████████████████████████████| 92kB 13.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
# 네이버 영화 리뷰 다운로드
!wget https://github.com/e9t/nsmc/raw/master/ratings.txt

import pandas as pd
import numpy as np
df = pd.read_csv("./ratings.txt",sep='\t').dropna()
df.head(5)

--2020-08-24 23:40:09--  https://github.com/e9t/nsmc/raw/master/ratings.txt
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt [following]
--2020-08-24 23:40:10--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘ratings.txt’

ratings.txt         100%[===================>]  18.61M  --.-KB/s    in 0.1s    

2020-08-24 23:40:11 (141 MB/s) - ‘ratings.txt’ saved [19515078/19515078]



,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


## 2) 전처리

In [ ]:
!wget https://www.dropbox.com/s/stt4y0zcp2c0iyb/ko.tar.gz
!tar xvzf ko.tar.gz

--2020-08-24 23:40:11--  https://www.dropbox.com/s/stt4y0zcp2c0iyb/ko.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/stt4y0zcp2c0iyb/ko.tar.gz [following]
--2020-08-24 23:40:12--  https://www.dropbox.com/s/raw/stt4y0zcp2c0iyb/ko.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccc6446f8516e88d64ecc9fe0eb.dl.dropboxusercontent.com/cd/0/inline/A-FcNH-78QhfzjCOJRS6mcWjZbTDw2MSORgS-C1ZNoo-O_4gNFsFBuFeuOJxNXCI6L1eiUCqdzULD_3na5X5z2fIAw0gOEYkkxqN9ThwWpg5XQ/file# [following]
--2020-08-24 23:40:12--  https://uccc6446f8516e88d64ecc9fe0eb.dl.dropboxusercontent.com/cd/0/inline/A-FcNH-78QhfzjCOJRS6mcWjZbTDw2MSORgS-C1ZNoo-O_4gNFsFBuFeuOJxNXCI6L1eiUCqdzULD_3na5X5z2fIAw0gOEYkkxqN9ThwWpg5XQ/file
Resolving uccc6446f8516e8

In [ ]:
import numpy as np

def load_dic(dic_file) :
  embeddings_index = dict()
  f = open(dic_file)
  for i, line in enumerate(f):
    if i == 0 : continue
    try :
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
    except :
      print(i)
      print(values)
  f.close()

  return embeddings_index

embeddings_index = load_dic('ko.vec')

In [ ]:
import json
from tensorflow.keras import preprocessing
import konlpy 
from konlpy.tag import Okt 

def preprocess(x, y, padding_size = 128, oov_token="<UNK>", vocab_file = "vocab.json", train_ratio=0.7) :  
  okt = Okt()
  X_test = [] 

  for sentence in x: 
    stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거 
    X_test.append(temp_X)

  preprocessor = preprocessing.text.Tokenizer(oov_token=oov_token) #토큰화
  preprocessor.fit_on_texts(X_test)
  x = preprocessor.texts_to_sequences(x) #시퀀스로 변환
  vocab = preprocessor.word_index #단어:인덱스 dictionary
  json.dump(vocab, open(vocab_file, 'w'), ensure_ascii=False)
  x = preprocessing.sequence.pad_sequences(x, maxlen=padding_size, padding='post', truncating='post')

  index = int(len(x)*train_ratio)

  return x[:index], y[:index], x[index:], y[index:], vocab

x_train, y_train, x_test, y_test, vocab = preprocess(df["document"].tolist(), df["label"].tolist())

In [ ]:
def gen_embedding_matrix(vocab, embeddings_index, embedding_size) :
  vocabulary_size = len(vocab)

  embedding_matrix = np.zeros((vocabulary_size, embedding_size))
  for word, index in vocab.items():
      if index > vocabulary_size - 1:
          break
      else:
          embedding_vector = embeddings_index.get(word)
          if embedding_vector is not None:
              embedding_matrix[index] = embedding_vector
  return embedding_matrix

embedding_matrix = gen_embedding_matrix(vocab, embeddings_index, embedding_size=200)

## 3) 모델 정의

In [ ]:
from tensorflow import keras

def CNNforText( num_classes,  #클래스 갯수
          vocab_size,
          embedding_matrix,           
          filter_sizes = [3, 4, 5],
          regularizers_lambda = 0.01, #L2 regulation parameter
          dropout =  0.5,
          feature_size = 128, #문장 시퀀스 길이
          num_filters = 128 #필터 개수 (필터사이즈와 같음). 
) :

  inputs = keras.Input(shape=(feature_size,), name='input_data')   
  embed_size = embedding_matrix.shape[1]
  embed = keras.layers.Embedding(vocab_size, embed_size, input_length=feature_size, weights=[embedding_matrix], trainable=False, name='embedding')(inputs)
  embed = keras.layers.Reshape((feature_size, embed_size, 1), name='add_channel')(embed)
  pool_outputs = []

  #filter 별로 모델 구성
  for filter_size in filter_sizes :
    #convolution
    filter_shape = (filter_size, embed_size)
    conv = keras.layers.Conv2D(num_filters, filter_shape, strides=(1, 1), padding='valid',
                                data_format='channels_last', activation='relu',
                                kernel_initializer='glorot_normal',
                                bias_initializer=keras.initializers.constant(0.1),
                                name='convolution_{:d}'.format(filter_size))(embed)
    #max pooling
    max_pool_shape = (feature_size - filter_size + 1, 1)
    pool = keras.layers.MaxPool2D(pool_size=max_pool_shape,
                                  strides=(1, 1), padding='valid',
                                  data_format='channels_last',
                                  name='max_pooling_{:d}'.format(filter_size))(conv)
    pool_outputs.append(pool)

  pool_outputs = keras.layers.concatenate(pool_outputs, axis=-1, name='concatenate')
  pool_outputs = keras.layers.Flatten(data_format='channels_last', name='flatten')(pool_outputs)
  pool_outputs = keras.layers.Dropout(dropout, name='dropout')(pool_outputs)

  outputs = keras.layers.Dense(num_classes, activation='softmax',
                                kernel_initializer='glorot_normal',
                                bias_initializer=keras.initializers.constant(0.1),
                                kernel_regularizer=keras.regularizers.l2(regularizers_lambda),
                                bias_regularizer=keras.regularizers.l2(regularizers_lambda),
                                name='dense')(pool_outputs)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.summary()
  return model, num_classes

model, num_classes = CNNforText(len(np.unique(y_train)), len(vocab), embedding_matrix)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 128, 200)     11341200    input_data[0][0]                 
__________________________________________________________________________________________________
add_channel (Reshape)           (None, 128, 200, 1)  0           embedding[0][0]                  
__________________________________________________________________________________________________
convolution_3 (Conv2D)          (None, 126, 1, 128)  76928       add_channel[0][0]                
_______________________________________________________________________________________

## 4) 학습

In [ ]:
import os
import time
import tensorflow as tf

def train(model, x_train, y_train, num_classes
          , batch_size = 64, epochs = 1, fraction_validation = 0.05, results_dir = "./result/", save_path = "model") :
  timestamp = time.strftime("%Y-%m-%d-%H-%M", time.localtime(time.time()))
  path = os.path.join(results_dir, timestamp)
  if not os.path.exists(path) :    
    path_log = os.path.join(path, 'log/')
    os.makedirs(path_log)

  model.compile(tf.optimizers.Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
  #모델 구조 이미지 파일로 저장
  keras.utils.plot_model(model, show_shapes=True, to_file=os.path.join(path, "model.jpg"))
  y_train = tf.one_hot(y_train, num_classes)
  tb_callback = keras.callbacks.TensorBoard(path_log,
                                            histogram_freq=0.1, write_graph=True,
                                            write_images=True,
                                            embeddings_freq=0.5, update_freq='batch')

  history = model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs,
                                callbacks=[tb_callback], validation_split=fraction_validation, shuffle=True)
  
  #모델 저장
  keras.models.save_model(model, save_path)
  print(history.history)

  return model, path_log

model, path_log = train(model, x_train, y_train, num_classes, epochs=1)

   1/2079 [..............................] - ETA: 1s - loss: 0.8216 - accuracy: 0.2656WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2079/2079 [==============================] - 22s 11ms/step - loss: 0.5261 - accuracy: 0.7651 - val_loss: 1.2333 - val_accuracy: 0.1396
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model/assets
{'loss': [0.5261073112487793], 'accuracy': [0.7651247382164001], 'val_loss': [1.2332805395126343], 'val_accuracy': [0.1395714282989502]}


In [ ]:
%load_ext tensorboard
%tensorboard --logdir {path_log}

## 5) 평가

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

def test(model, x_test, y_test, num_classes):
  y_pred_one_hot = model.predict(x=x_test, batch_size=1, verbose=1)
  y_pred = tf.math.argmax(y_pred_one_hot, axis=1)

  print('\nTest accuracy: {}\n'.format(accuracy_score(y_test, y_pred)))
  print('Classification report:')
  target_names = ['class {:d}'.format(i) for i in np.arange(num_classes)]
  print(classification_report(y_test, y_pred, target_names=target_names, digits=4))

test(model, x_test[:50000], y_test[:50000], num_classes)